# DTOcean Moorings and Foundation Test

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [4]:
import numpy as np

In [5]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu
from dtocean_core.pipeline import Tree

In [6]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [7]:
# Bring up the logger
start_logging()

2016-09-23 16:11:34,483 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [8]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
pipe_tree = Tree()

## Create a new project

In [9]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2016-09-23 16:11:39,063 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2016-09-23 16:11:39,063 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2016-09-23 16:11:39,063 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [10]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type" 
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Tidal Floating")
my_var.read(new_core, new_project)

2016-09-23 16:11:39,084 - INFO - aneris.control.data - New "device.system_type" data stored with index R8KCJH
2016-09-23 16:11:39,084 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [11]:
project_menu.initiate_pipeline(new_core, new_project)

2016-09-23 16:11:39,104 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2016-09-23 16:11:39,104 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index YK9XIB
2016-09-23 16:11:39,104 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2016-09-23 16:11:39,104 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2016-09-23 16:11:39,104 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [12]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [13]:
module_name = 'Mooring and Foundations'
module_menu.activate(new_core, new_project, module_name)

## Check the status of the module inputs

In [14]:
mooring_branch = pipe_tree.get_branch(new_core, new_project, 'Mooring and Foundations')
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [15]:
project_menu.initiate_dataflow(new_core, new_project)

2016-09-23 16:11:39,223 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index XC9SCH
2016-09-23 16:11:39,233 - INFO - aneris.control.simulation - Datastate stored
2016-09-23 16:11:39,243 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [16]:
%run inputs_wp4.py

generate test data: D:\Usuarios\108630\Programming\Python\notebooks\inputs_wp4.pkl


In [17]:
mooring_branch.read_test_data(new_core,
                              new_project,
                              "inputs_wp4.pkl")

2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "farm.mean_wind_speed_100_year" data stored with index T54Z5E
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "device.system_profile" data stored with index DMPPYC
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "farm.mooring_ALS_safety_factor" data stored with index YFTD8C
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "farm.mean_wind_direction_100_year" data stored with index MB3ST7
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "device.umbilical_safety_factor" data stored with index EJDIT2
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "device.turbine_diameter" data stored with index 3BD3J3
2016-09-23 16:12:00,783 - INFO - aneris.control.data - New "device.system_mass" data stored with index 5QEAZL
2016-09-23 16:12:00,793 - INFO - aneris.control.data - New "device.turbine_performance" data stored with index 4JC16U
2016-09-23 16:12:00,793 - INFO - aneris.control.data -

## Check if the module can be executed

In [18]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

True

In [19]:
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [20]:
module_menu.execute_current(new_core, new_project)

2016-09-23 16:12:00,904 - INFO - aneris.control.simulation - Datastate with level "mooring and foundations start" stored
          presubstfound   submass  subvol  subwetfa  subwetba  subdryfa  \
substid                                                                   
array                []  500000.0  281.25      37.5      37.5       0.0   
subhub001            []  100000.0   50.00      10.0      10.0       0.0   
subhub002            []  100000.0   50.00      10.0      10.0       0.0   

           subdryba  sublength  subwidth  subheight      subprof  subrough  \
substid                                                                      
array           0.0        7.5       7.5        5.0  rectangular  0.000005   
subhub001       0.0        5.0       5.0        2.0  rectangular  0.000005   
subhub002       0.0        5.0       5.0        2.0  rectangular  0.000005   

           suborienang                     suborig           subcog  \
substid                                  

## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [21]:
output_status = mooring_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [22]:
economics = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_foundations_economics_data")
economics.get_value(new_core, new_project)

,Key Identifier,Quantity,Cost,Year
0,id743,1.0,30989.037383,0.0
1,id471,1.0,1202.730000,0.0
2,id316,1.0,67.500000,0.0
3,id68,1.0,493.506859,0.0
4,id316,1.0,67.500000,0.0
5,id422,1.0,2730.274127,0.0
6,id316,1.0,67.500000,0.0
7,id338,1.0,646.040000,0.0
8,id316,1.0,67.500000,0.0
9,id471,1.0,1202.730000,0.0


In [23]:
moorings = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_component_data")
moorings.get_value(new_core, new_project)

,Line Identifier,Marker
0,line000,0
1,line000,1
2,line000,2
3,line000,3
4,line000,4
5,line000,5
6,line000,6
7,line000,7
8,line001,8
9,line001,9


In [24]:
lines = mooring_branch.get_output_variable(new_core, new_project, "farm.line_data")
lines.get_value(new_core, new_project)

,Line Identifier,Type,Length,Dry Mass
0,line000,catenary,219.174306,1141.791230
1,line001,catenary,220.467777,1127.139780
2,line002,catenary,212.340890,1223.509418
3,line003,catenary,205.518546,1313.580597


In [25]:
umbilicals = mooring_branch.get_output_variable(new_core, new_project, "farm.umbilical_cable_data")
umbilicals.get_value(new_core, new_project)

,Key Identifier,Marker,Length,Dry Mass,Required Floatation,Floatation Length
0,id743,0032,619.780748,6817.588224,151.02241,123.95615


In [26]:
dimensions = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_dimensions")
dimensions.get_value(new_core, new_project)

,Footprint Area,System Identifier,Volume
0,0.000000,subhub001,0.000000e+00
1,15707.963268,device001,1.898742e+06
2,0.000000,subhub002,0.000000e+00
3,0.000000,array,0.000000e+00


In [27]:
network = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_foundations_network")
net_val = network.get_value(new_core, new_project)

In [28]:
import pprint
pprint.pprint(net_val)

{'nodes': {'array': {'Substation foundation': {'marker': [[41, 42]],
                                               'quantity': Counter({'id718': 1, 'grout': 1})}},
           'device001': {'Foundation': {'marker': [[33, 34],
                                                   [35, 36],
                                                   [37, 38],
                                                   [39, 40]],
                                        'quantity': Counter({'id516': 4, 'grout': 4})},
                         'Mooring system': {'marker': [[0,
                                                        1,
                                                        2,
                                                        3,
                                                        4,
                                                        5,
                                                        6,
                                                        7],
                             